In [1]:
!git clone https://github.com/minyoung347/highfrequency.git

import sys
sys.path.insert(0,'/content/highfrequency')

Cloning into 'highfrequency'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 46 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (46/46), 6.39 MiB | 1.26 MiB/s, done.


In [2]:
import typing
from typing import Tuple
import json
import os
import sys

import torch
from torch import nn
from torch import optim
from sklearn.preprocessing import StandardScaler
import joblib

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import utils
from modules import Encoder, Decoder
from custom_types import DaRnnNet, TrainData, TrainConfig
from utils import numpy_to_tensor
from constants import device

logger = utils.setup_log()
logger.info(f"Using computation device: {device}")

2023-06-11 15:50:46,750 - VOC_TOPICS - INFO - Using computation device: cpu


In [3]:
device

device(type='cpu')

In [ ]:
def preprocess_data(dat, col_names, scale=None) -> Tuple[TrainData, StandardScaler]:
    if scale is None:
        scale = StandardScaler().fit(dat)
    proc_dat = scale.transform(dat)
    proc_dat = np.array(proc_dat)  # 2023
    # proc_dat = np.array(dat)  # 2023
    mask = np.ones(proc_dat.shape[1], dtype=bool)
    dat_cols = list(dat.columns)
    for col_name in col_names:
        mask[dat_cols.index(col_name)] = False

    feats = proc_dat[:, mask]
    targs = proc_dat[:, ~mask]

    return TrainData(feats, targs), scale


def da_rnn(train_data: TrainData, n_targs: int, encoder_hidden_size=50, decoder_hidden_size=50, # 50
           T=10, learning_rate=0.001, batch_size=128):

    train_cfg = TrainConfig(T, int(train_data.feats.shape[0] * 0.7), batch_size, nn.MSELoss())
    logger.info(f"Training size: {train_cfg.train_size:d}.")

    enc_kwargs = {"input_size": train_data.feats.shape[1], "hidden_size": encoder_hidden_size, "T": T}
    encoder = Encoder(**enc_kwargs).to(device)
    with open(os.path.join("data", "enc_kwargs.json"), "w") as fi:
        json.dump(enc_kwargs, fi, indent=4)

    dec_kwargs = {"encoder_hidden_size": encoder_hidden_size,
                  "decoder_hidden_size": decoder_hidden_size, "T": T, "out_feats": n_targs}
    decoder = Decoder(**dec_kwargs).to(device)
    with open(os.path.join("data", "dec_kwargs.json"), "w") as fi:
        json.dump(dec_kwargs, fi, indent=4)

    encoder_optimizer = optim.Adam(
        params=[p for p in encoder.parameters() if p.requires_grad],
        lr=learning_rate, weight_decay = 1e-6) # weight_decay = 1e-5
    decoder_optimizer = optim.Adam(
        params=[p for p in decoder.parameters() if p.requires_grad],
        lr=learning_rate, weight_decay = 1e-6)
    da_rnn_net = DaRnnNet(encoder, decoder, encoder_optimizer, decoder_optimizer)

    return train_cfg, da_rnn_net


def train(net: DaRnnNet, train_data: TrainData, t_cfg: TrainConfig, n_epochs=10, save_plots=True):
    iter_per_epoch = int(np.ceil(t_cfg.train_size * 1. / t_cfg.batch_size))
    iter_losses = np.zeros(n_epochs * iter_per_epoch)
    epoch_losses = np.zeros(n_epochs)
    iter_losses_val = np.zeros(n_epochs * iter_per_epoch) #
    epoch_losses_val = np.zeros(n_epochs) #
    logger.info(f"Iterations per epoch: {t_cfg.train_size * 1. / t_cfg.batch_size:3.3f} ~ {iter_per_epoch:d}.")

    n_iter = 0

    for e_i in range(n_epochs):
        perm_idx = np.random.permutation(t_cfg.train_size - t_cfg.T)

        t_iter = 0
        for t_i in range(0, t_cfg.train_size, t_cfg.batch_size):
            batch_idx = perm_idx[t_i : t_i + t_cfg.batch_size]
            feats, y_history, y_target = prep_train_data(batch_idx, t_cfg, train_data)

            loss, attn_weight = train_iteration(net, t_cfg.loss_func, feats, y_history, y_target)
            if t_iter == 0:
                torch.save(attn_weight, './pt_save/attn_w_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.pt'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size, e_i, t_iter))
                torch.save(y_target, './pt_save/y_target_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.pt'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size, e_i, t_iter))
                y_test_pred = predict(net, train_data, #
                                  t_cfg.train_size, t_cfg.batch_size, t_cfg.T, #
                                  on_train=False) #
                torch.save(y_test_pred, './pt_save/y_pred_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.pt'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size, e_i, t_iter))
            
            y_test_pred = predict(net, train_data, #
                                  t_cfg.train_size, t_cfg.batch_size, t_cfg.T, #
                                  on_train=False) #
            # TODO: make this MSE and make it work for multiple inputs
            val_loss = y_test_pred - train_data.targs[t_cfg.train_size:] #
            iter_losses_val[e_i * iter_per_epoch + t_i // t_cfg.batch_size] = np.mean(np.abs(val_loss)) #
            
            
            iter_losses[e_i * iter_per_epoch + t_i // t_cfg.batch_size] = loss
            # if (j / t_cfg.batch_size) % 50 == 0:
            #    self.logger.info("Epoch %d, Batch %d: loss = %3.3f.", i, j / t_cfg.batch_size, loss)
            
            file_iter_loss = open('./loss_save/iter_loss_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size),'a')
            file_iter_loss.write('{}, {}, {}, {}\n'.format(e_i, n_iter, loss, np.mean(np.abs(val_loss))))
            file_iter_loss.close()
            
            n_iter += 1

            adjust_learning_rate(net, n_iter)
            
            t_iter += 1

        epoch_losses[e_i] = np.mean(iter_losses[range(e_i * iter_per_epoch, (e_i + 1) * iter_per_epoch)])
        epoch_losses_val[e_i] = np.mean(iter_losses_val[range(e_i * iter_per_epoch, (e_i + 1) * iter_per_epoch)])
        
        file_epoch_loss = open('./loss_save/epoch_loss_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size),'a')
        file_epoch_loss.write('{}, {}, {}, {}\n'.format(e_i, n_iter, np.mean(iter_losses[range(e_i * iter_per_epoch, (e_i + 1) * iter_per_epoch)]), np.mean(np.abs(val_loss)), np.mean(iter_losses_val[range(e_i * iter_per_epoch, (e_i + 1) * iter_per_epoch)])))
        file_epoch_loss.close()
        

        if e_i % 10 == 0:
            y_test_pred = predict(net, train_data,
                                  t_cfg.train_size, t_cfg.batch_size, t_cfg.T,
                                  on_train=False)
            # TODO: make this MSE and make it work for multiple inputs
            val_loss = y_test_pred - train_data.targs[t_cfg.train_size:]
            #logger.info(f"Epoch {e_i:d}, niter {n_iter:d}, train loss: {epoch_losses[e_i]:3.3f}, val loss: {np.mean(np.abs(val_loss))}.")
            logger.info(f"Epoch {e_i:d}, niter {n_iter:d}, train loss: {epoch_losses[e_i]}, val loss: {np.mean(np.abs(val_loss))}.")
            
            
            y_train_pred = predict(net, train_data,
                                   t_cfg.train_size, t_cfg.batch_size, t_cfg.T,
                                   on_train=True)

            plt.figure(figsize=(10,7))
            #plt.subplot(1,3,1)
            plt.plot(range(1, 1 + len(train_data.targs)), train_data.targs,
                     'r', label="True")
            plt.plot(range(t_cfg.T, len(y_train_pred) + t_cfg.T), y_train_pred,
                     'g' ,label='Predicted - Train')
            plt.plot(range(t_cfg.T + len(y_train_pred), len(train_data.targs) + 1), y_test_pred,
                     'b', label='Predicted - Test')
            plt.legend(loc='upper left')
            #plt.show()
            #plt.subplot(1,3,2)
            #plt.scatter(train_data.targs[:len(y_train_pred)], y_train_pred)
            #plt.subplot(1,3,3)
            #plt.scatter(train_data.targs[len(y_train_pred):], y_test_pred)
            utils.save_or_show_plot("pred_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.png".format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size, e_i), save_plots)
            plt.close()
            
            '''
            correct_case = 0
            uncorrect_case = 0
            for i, ii in zip(train_data.targs[:len(y_train_pred)], y_train_pred):
                if i != 0:
                    if np.sign(i) == np.sign(ii):
                        correct_case += 1
                    else:
                        uncorrect_case += 1
            acc_train = correct_case / (correct_case + uncorrect_case)
            
            correct_case = 0
            uncorrect_case = 0
            for i, ii in zip(train_data.targs[len(y_train_pred):], y_test_pred):
                if i != 0:
                    if np.sign(i) == np.sign(ii):
                        correct_case += 1
                    else:
                        uncorrect_case += 1
            acc_test = correct_case / (correct_case + uncorrect_case)
            
            file = open('./pred_save/acc_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs),'a')
            file.write('{}, {}\n'.format(e_i, n_iter, acc_train, acc_test))
            file.close()
            '''
           
    return iter_losses, epoch_losses, iter_losses_val, epoch_losses_val


def prep_train_data(batch_idx: np.ndarray, t_cfg: TrainConfig, train_data: TrainData):
    feats = np.zeros((len(batch_idx), t_cfg.T - 1, train_data.feats.shape[1]))
    y_history = np.zeros((len(batch_idx), t_cfg.T - 1, train_data.targs.shape[1]))
    y_target = train_data.targs[batch_idx + t_cfg.T]

    for b_i, b_idx in enumerate(batch_idx):
        start, stop = b_idx, b_idx + t_cfg.T - 1
        feats[b_i, :, :] = train_data.feats[start : stop, :]
        y_history[b_i, :] = train_data.targs[start : stop]

    return feats, y_history, y_target


def adjust_learning_rate(net: DaRnnNet, n_iter: int):
    # TODO: Where did this Learning Rate adjustment schedule come from?
    # Should be modified to use Cosine Annealing with warm restarts https://www.jeremyjordan.me/nn-learning-rate/
    #if n_iter % 10000 == 0 and n_iter > 0:
    if n_iter % 1000 == 0 and n_iter > 0:
        for enc_params, dec_params in zip(net.enc_opt.param_groups, net.dec_opt.param_groups):
            enc_params['lr'] = enc_params['lr'] * 0.9
            dec_params['lr'] = dec_params['lr'] * 0.9


def train_iteration(t_net: DaRnnNet, loss_func: typing.Callable, X, y_history, y_target):
    t_net.enc_opt.zero_grad()
    t_net.dec_opt.zero_grad()

    input_weighted, input_encoded, attn_w = t_net.encoder(numpy_to_tensor(X))
    y_pred = t_net.decoder(input_encoded, numpy_to_tensor(y_history))

    y_true = numpy_to_tensor(y_target)
    loss = loss_func(y_pred, y_true)
    loss.backward()

    t_net.enc_opt.step()
    t_net.dec_opt.step()

    return loss.item(), attn_w


def predict(t_net: DaRnnNet, t_dat: TrainData, train_size: int, batch_size: int, T: int, on_train=False):
    out_size = t_dat.targs.shape[1]
    if on_train:
        y_pred = np.zeros((train_size - T + 1, out_size))
    else:
        y_pred = np.zeros((t_dat.feats.shape[0] - train_size, out_size))

    for y_i in range(0, len(y_pred), batch_size):
        y_slc = slice(y_i, y_i + batch_size)
        batch_idx = range(len(y_pred))[y_slc]
        b_len = len(batch_idx)
        X = np.zeros((b_len, T - 1, t_dat.feats.shape[1]))
        y_history = np.zeros((b_len, T - 1, t_dat.targs.shape[1]))

        for b_i, b_idx in enumerate(batch_idx):
            if on_train:
                start, stop = b_idx, b_idx + T - 1
            else:
                start, stop = b_idx + train_size - T, b_idx + train_size - 1

            X[b_i, :, :] = t_dat.feats[start : stop, :]
            y_history[b_i, :] = t_dat.targs[start : stop]

        y_history = numpy_to_tensor(y_history)
        _, input_encoded, _ = t_net.encoder(numpy_to_tensor(X))
        y_pred[y_slc] = t_net.decoder(input_encoded, y_history).cpu().data.numpy()

    return y_pred


In [ ]:
save_plots = True
debug = False
    
firm_num = 1
start_day = 1
end_day = 169
add_random = 0
# num_variables = 81 # exclude target variable
num_variables = 90 # exclude target variable
lr = 0.01
num_epoch = 100
bs = 1024
tc = 10
ehs = 100
dhs = 100
agg_size = 100

In [ ]:

if os.path.exists('./loss_save/iter_loss_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size)):
    os.remove('./loss_save/iter_loss_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size))
if os.path.exists('./loss_save/epoch_loss_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size)):
    os.remove('./loss_save/epoch_loss_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size))
if os.path.exists('./pred_save/final_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size)):
    os.remove('./pred_save/final_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size))


if start_day == end_day:
    if add_random == 0:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}.csv".format(firm_num, start_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 1:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_r.csv".format(firm_num, start_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 2:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_re.csv".format(firm_num, start_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 3:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_re_r.csv".format(firm_num, start_day, agg_size)), nrows=100 if debug else None) 
    elif add_random == 4:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_all.csv".format(firm_num, start_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 5:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_all_r.csv".format(firm_num, start_day, agg_size)), nrows=100 if debug else None)    
else:
    if add_random == 0:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_{}.csv".format(firm_num, start_day, end_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 1:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_{}_r.csv".format(firm_num, start_day, end_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 2:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_{}_re.csv".format(firm_num, start_day, end_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 3:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_{}_re_r.csv".format(firm_num, start_day, end_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 4:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_{}_all.csv".format(firm_num, start_day, end_day, agg_size)), nrows=100 if debug else None)
    elif add_random == 5:
        raw_data = pd.read_csv(os.path.join("data/LSE_orderbook", "da_f{}_{}_{}_{}_all_r.csv".format(firm_num, start_day, end_day, agg_size)), nrows=100 if debug else None)


# raw_data = pd.read_csv('nasdaq100_padding.csv')
# raw_data.columns = names=['var' + str(i) for i in range(1,83)]
# raw_data = raw_data.iloc[0:2000]
    
    
logger.info(f"Shape of data: {raw_data.shape}.\nMissing in data: {raw_data.isnull().sum().sum()}.")
if add_random == 1 or add_random == 3 or add_random == 5:
    targ_cols = ("var{}".format(2*num_variables + 1),)
else:
    targ_cols = ("var{}".format(num_variables + 1),)
data, scaler = preprocess_data(raw_data, targ_cols)

2023-06-11 23:37:56,724 - VOC_TOPICS - INFO - Shape of data: (14349, 91).
Missing in data: 0.


In [ ]:
da_rnn_kwargs = {"batch_size": bs, "T": tc, "encoder_hidden_size": ehs, "decoder_hidden_size": dhs} # 1024, 10, 50, 50
config, model = da_rnn(data, n_targs=len(targ_cols), learning_rate=lr, **da_rnn_kwargs) #.01

2023-06-11 23:37:59,938 - VOC_TOPICS - INFO - Training size: 10044.


In [ ]:
iter_loss, epoch_loss, iter_loss_val, epoch_loss_val = train(model, data, config, n_epochs=num_epoch, save_plots=save_plots)
final_y_pred = predict(model, data, config.train_size, config.batch_size, config.T)

2023-06-11 23:38:02,352 - VOC_TOPICS - INFO - Iterations per epoch: 9.809 ~ 10.
2023-06-11 23:38:04,766 - VOC_TOPICS - INFO - Epoch 0, niter 10, train loss: 0.8997158288955689, val loss: 0.8396776787280622.
2023-06-11 23:38:28,107 - VOC_TOPICS - INFO - Epoch 10, niter 110, train loss: 0.8721830427646637, val loss: 0.8359963524105062.
2023-06-11 23:38:49,742 - VOC_TOPICS - INFO - Epoch 20, niter 210, train loss: 0.8641083598136902, val loss: 0.8353187410807841.
2023-06-11 23:39:11,251 - VOC_TOPICS - INFO - Epoch 30, niter 310, train loss: 0.8331318795681, val loss: 0.8472448199834722.
2023-06-11 23:39:32,812 - VOC_TOPICS - INFO - Epoch 40, niter 410, train loss: 0.804358696937561, val loss: 0.8628435953636514.
2023-06-11 23:39:54,386 - VOC_TOPICS - INFO - Epoch 50, niter 510, train loss: 0.7447859704494476, val loss: 0.8823428216192687.
2023-06-11 23:40:15,992 - VOC_TOPICS - INFO - Epoch 60, niter 610, train loss: 0.6742965817451477, val loss: 0.9044516389674948.
2023-06-11 23:40:38,585

In [ ]:
plt.figure()
plt.semilogy(range(len(iter_loss)), iter_loss, label='train loss')
plt.semilogy(range(len(iter_loss_val)), iter_loss_val, label='valiation loss')
utils.save_or_show_plot("iter_loss_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.png".format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size), save_plots)
plt.close()

plt.figure()
plt.semilogy(range(len(epoch_loss)), epoch_loss, label='epoch train loss')
plt.semilogy(range(len(epoch_loss_val)), epoch_loss_val, label='epoch valiadation loss')
utils.save_or_show_plot("epoch_loss_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.png".format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size), save_plots)
plt.close()

plt.figure()
plt.plot(final_y_pred, label='Predicted')
plt.plot(data.targs[config.train_size:], label="True")
plt.legend(loc='upper left')
utils.save_or_show_plot("final_predicted_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.png".format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size), save_plots)
plt.close()


correct_case = 0
uncorrect_case = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i, ii in zip(data.targs[config.train_size:], final_y_pred):
    if i[0] != 0:
        if np.sign(i[0]) == np.sign(ii[0]):
            correct_case += 1
        else:
            uncorrect_case += 1
            
        if np.sign(i[0]) > 0:
            if np.sign(ii[0]) > 0:
                TP += 1
            elif np.sign(ii[0]) < 0:
                FN += 1
        elif np.sign(i[0]) < 0: 
            if np.sign(ii[0]) > 0:
                FP += 1
            elif np.sign(ii[0]) < 0:
                TN += 1
                            
acc_final = correct_case / (correct_case + uncorrect_case)
print(acc_final)


file_stat = open('./stat_save/stat_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size),'a')
file_stat.write('{}, {}, {}, {}, {}\n'.format(acc_final, TP, FN, FP, TN))
file_stat.close()



file_iter_loss = open('./pred_save/final_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.csv'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size),'a')
for i, ii in zip(data.targs[config.train_size:], final_y_pred):
    file_iter_loss.write('{}, {}\n'.format(i[0], ii[0]))
file_iter_loss.close()

with open(os.path.join("data", "da_rnn_kwargs.json"), "w") as fi:
    json.dump(da_rnn_kwargs, fi, indent=4)

joblib.dump(scaler, os.path.join("data", "scaler.pkl"))
torch.save(model.encoder.state_dict(), os.path.join("data", "encoder.torch"))
torch.save(model.decoder.state_dict(), os.path.join("data", "decoder.torch"))

0.675493612078978


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt


firm_num = 1
start_day = 1
end_day = 169
add_random = 0
# num_variables = 81 # exclude target variable
num_variables = 90 # exclude target variable
lr = 0.01
num_epoch = 10
bs = 1024
tc = 10
ehs = 100
dhs = 100
agg_size = 100

e_i = 1
t_iter = 0

attn_w = torch.load('./pt_save/attn_w_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.pt'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size, e_i, t_iter))
attn_w = attn_w.detach().cpu().numpy()

y_target = torch.load('./pt_save/y_target_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.pt'.format(firm_num, start_day, end_day, add_random, num_variables, lr, num_epoch, bs, tc, ehs, dhs, agg_size, e_i))

In [ ]:
print(np.shape(attn_w))
print(np.shape(y_target))

(1024, 9, 90)
(1024, 1)


In [ ]:
# attn_positive = np.zeros(62*2)
# attn_negative = np.zeros(62*2)
attn_positive = np.zeros(num_variables)
attn_negative = np.zeros(num_variables)

print(np.shape(attn_w))
print(np.shape(y_target))

for i in range(len(y_target)-1):
    if y_target[i+1] - y_target[i] > 0:
        attn_positive += np.mean(attn_w[i+1], axis = 0)
    elif y_target[i+1] - y_target[i] < 0:
        attn_negative += np.mean(attn_w[i+1], axis = 0)

(1024, 9, 90)
(1024, 1)


ValueError: operands could not be broadcast together with shapes (81,) (90,) (81,) 

In [ ]:
plt.plot(attn_positive)
plt.savefig('test.png')